## create synthe grid

In [1]:
# because Jupyter notebook is not in package
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
from vidmapy.kurucz.atlas import Atlas
from vidmapy.kurucz.synthe import Synthe
from vidmapy.kurucz.parameters import Parameters

In [4]:
atlas_worker= Atlas()
synthe_worker= Synthe()

In [5]:
def get_atlas_model(teff, logg, feh, vmicro=2.):
    p = Parameters(teff=teff, logg=logg, metallicity=feh, microturbulence=vmicro)
    return atlas_worker.get_model(p)

def get_synthe_model(teff, logg, feh, wmin, wmax, resolution, vmicro=2.):
    model = get_atlas_model(teff, logg, feh)
    p_synthe = Parameters(wave_min=wmin, wave_max=wmax, resolution=resolution, metallicity=feh)
    spectrum = synthe_worker.get_spectrum(model, parameters=p_synthe, quiet=False)
    return model, spectrum

[Back to the top](#VidmaPy-examples)

<a id='ex4'></a>
## Parallelization of spectra computation

In [6]:
import multiprocessing as mp
import pickle
import pandas as pd

In [7]:
teff = np.arange(3500, 7250, 250)
logg = np.arange(1., 5.5, 0.5)
feh = np.arange(-1, 0.500001, 0.25)
resolution = 200000.

In [ ]:
models = []
for _teff in teff:
    for _logg in logg:
        for _feh in feh:
            print (_teff, _logg, _feh)
            models.append(get_atlas_model(_teff, _logg, _feh))

In [ ]:
with open("models.pkl", "wb") as f:
    pickle.dump(models, f)

In [ ]:
pvals = []
for _teff in teff:
    for _logg in logg:
        for _feh in feh:
            pvals.append((_teff, _logg, _feh))
params = pd.DataFrame(pvals, columns=['teff', 'logg', 'feh'])
params['resolution'] = resolution
params['vmic'] = 2.0

In [ ]:
params.to_csv("model_params.csv", index=False)